# Generate a RestAPI for Metadata with Django

Philipp S. Sommer

Helmholtz Coastal Data Center (HCDC)
Helmholtz-Zentrum Geesthacht, Institute of Coastal Research

## What's the problem with CSW?

- it's XML!
- too complicated to use for scientists or web development

&rarr; Provide metadata through RestAPI

### Examples:
- [CERA (DKRZ)](https://cera-www.dkrz.de/WDCC/ui/cerasearch/cerarest/entry?acronym=DKRZ_LTA_706_ds00002)
- [O2A (AWI)](https://dashboard.awi.de/de.awi.data.ws/api/)

## What is a RestAPI

- [official definition at restfulapi.net](https://restfulapi.net/)
- common implementation:
    - provide JSON-formatted data (or other formats) via `GET` request
    - alter data on the server via `PUT` request
    - do something via `POST` request

## Purpose of this talk

Provide a simple and basic example from scratch to show the functionality of serving metadata via RestAPI.

### Requirements to run this notebook

- `linux` or `osx`
- `django`
- `djangorestframework`, for the rest api
- `uritemplate`, for generating an openAPI schema

and for generating a graph of the database_
- `graphviz`
- `django-extensions`

Install everything via:

```bash
conda create -n django-metadata -c conda-forge django-extensions graphviz uritemplate djangorestframework
```

In [ ]:
rm -r django-metadata-api

## Initialize the django-metadata-api project

In [ ]:
!mkdir django-metadata-api
!django-admin startproject django_metadata_api django-metadata-api

In [ ]:
cd django-metadata-api

### Create a django app in this project

We call it `api`. Here we will do all our work.

In [ ]:
!python manage.py startapp api

## Make the first migration

This will generate an sqlite3 database (but we could also use something else...)

In [ ]:
!python manage.py migrate

## The django project structure

In [ ]:
!tree 

## The project settings

In [ ]:
!cat django_metadata_api/settings.py

## Add the necessary apps to the settings

In [ ]:
%%writefile -a django_metadata_api/settings.py

INSTALLED_APPS += [
    "api",
    "rest_framework",
    "django_extensions",
]

## Django models

Each `Model` (inherits the `django.db.models.Model` class) defines a table in our (sqlite3) database. The `fields` of each model correspond to the columns of the database table.

Initially, there are no models defined.

In [ ]:
!cat api/models.py

## Creating models

So let's define some.

In [ ]:
%%writefile api/models.py

from django.db import models


class Institution(models.Model):
    """A research institution."""

    name = models.CharField(
        max_length=250,
        help_text="Name of the institution",
    )
    
    abbreviation = models.CharField(
        max_length=10, 
        help_text="Abbreviation of the institution"
    )
    
    def __str__(self):
        return f"{self.name} ({self.abbreviation})"


## Django models

and some more

In [ ]:
%%writefile -a api/models.py

class Person(models.Model):
    """A person."""
    
    first_name = models.CharField(
        max_length=50,
        help_text="First name of the person"
    )
    
    last_name = models.CharField(
        max_length=50,
        help_text="Last name of the person"
    )
    
    email = models.EmailField(
        max_length=255,
        help_text="Email address of the person.",
    )
    
    institution = models.ForeignKey(
        Institution,
        on_delete=models.PROTECT,
        help_text="Research institution of the person."
    )
    
    def __str__(self):
        return f"{self.first_name} {self.last_name} ({self.institution.abbreviation})"


## Django models

and some more

In [ ]:
%%writefile -a api/models.py
    
class Project(models.Model):
    """A research project."""
    
    name = models.CharField(
        max_length=250,
        help_text="Full name of the project",
    )
    
    abbreviation = models.CharField(
        max_length=50,
        help_text="Abbreviation of the project."
    )
    
    pi = models.ForeignKey(
        Person,
        on_delete=models.PROTECT,
        help_text="Principal investigator of the model."
    )
    
    def __str__(self):
        return f"{self.name} ({self.abbreviation})"

## Django models

and some more

In [ ]:
%%writefile -a api/models.py

class Dataset(models.Model):
    """A dataset output of a model."""
        
    class DataSource(models.TextChoices):
        """Available data sources."""

        model = "MODEL", "derived from a climate model"
        satellite = "SATELLITE", "derived from satellite observation"

    name = models.CharField(
        max_length=200,
        help_text="Name of the dataset."
    )
    
    source_type = models.CharField(
        max_length=20,
        choices=DataSource.choices,
        help_text="How the data has been derived."
    )
    
    project = models.ForeignKey(
        Project,
        on_delete=models.CASCADE,
        help_text="The project this dataset belongs to."
    )
    
    contact = models.ForeignKey(
        Person,
        on_delete=models.PROTECT,
        help_text="The contact person for this dataset",
    )

    def __str__(self):
        return f"{self.name} ({self.project.abbreviation})"

## Django models

and some more

In [ ]:
%%writefile -a api/models.py

class Parameter(models.Model):
    """A standardized parameter in our database."""

    name = models.CharField(
        max_length=200,
        help_text="Name of the dataset."
    )
    
    unit = models.CharField(
        max_length=20,
        help_text="Units of the parameter."
    )
    
    long_name = models.CharField(
        max_length=250,
        help_text="Description of the parameter"
    )
    
    dataset = models.ForeignKey(
        Dataset,
        help_text="The dataset that contains this parameter",
        related_name="parameters",
        on_delete=models.CASCADE,
    )

    def __str__(self):
        return f"{self.name} ({self.unit})"

## Getting an overview

`django-extensions` provide the functionality to show a graph of our models. So let's do this

In [ ]:
!python manage.py graph_models api > apigraph.dot
!dot apigraph.dot -Tsvg -o apigraph.svg

In [ ]:
from IPython.display import SVG
SVG(filename="apigraph.svg")

## Update the database

So far, we just wrote some python. Now tell Django to register our models in the (sqlite3) database:

In [ ]:
!python manage.py makemigrations  # creates the migration scripts

In [ ]:
!python manage.py migrate  # creates the tables in the database

## Add serializers to our models

A serializer transforms your model into JSON (and more).

In [ ]:
%%writefile api/serializers.py

from rest_framework import serializers
from api import models


class InstitutionSerializer(serializers.HyperlinkedModelSerializer):
    
    class Meta:
        model = models.Institution
        fields = '__all__'


## And serializers for the other models

In [ ]:
%%writefile -a api/serializers.py

class PersonSerializer(serializers.HyperlinkedModelSerializer):
    
    class Meta:
        model = models.Person
        fields = '__all__'


class ProjectSerializer(serializers.HyperlinkedModelSerializer):
    
    class Meta:
        model = models.Project
        fields = '__all__'


class DatasetSerializer(serializers.HyperlinkedModelSerializer):
    
    class Meta:
        model = models.Dataset
        fields = '__all__'


class ParameterSerializer(serializers.HyperlinkedModelSerializer):
    
    class Meta:
        model = models.Parameter
        fields = '__all__'

## Generate the viewset for the models

A viewset (comparable to an HTML webpage) tells django, you to display and update the serialized models.

In [ ]:
%%writefile api/views.py

from rest_framework import viewsets
from rest_framework import permissions

from api import models, serializers


class InstitutionViewSet(viewsets.ModelViewSet):
    """View the institutions"""
    
    queryset = models.Institution.objects.all()
    serializer_class = serializers.InstitutionSerializer


## And viewsets for the other models

In [ ]:
%%writefile -a api/views.py

class PersonViewSet(viewsets.ModelViewSet):
    """View the institutions"""
    
    queryset = models.Person.objects.all()
    serializer_class = serializers.PersonSerializer
    
    
class ProjectViewSet(viewsets.ModelViewSet):
    """View the institutions"""
    
    queryset = models.Project.objects.all()
    serializer_class = serializers.ProjectSerializer


class DatasetViewSet(viewsets.ModelViewSet):
    """View the institutions"""
    
    queryset = models.Dataset.objects.all()
    serializer_class = serializers.DatasetSerializer


class ParameterViewSet(viewsets.ModelViewSet):
    """View the institutions"""
    
    queryset = models.Parameter.objects.all()
    serializer_class = serializers.ParameterSerializer


## Define the router

We generated the webpages, but did not tell anything about where to find them.

This is the job of the `router`.

In [ ]:
%%writefile api/urls.py

from django.urls import include, path
from rest_framework import routers
from api import views

router = routers.DefaultRouter()
router.register(r'institutions', views.InstitutionViewSet)
router.register(r'persons', views.PersonViewSet)
router.register(r'projects', views.ProjectViewSet)
router.register(r'datasets', views.DatasetViewSet)
router.register(r'parameters', views.ParameterViewSet)

# Wire up our API using automatic URL routing.
# Additionally, we include login URLs for the browsable API.
urlpatterns = [
    path('', include(router.urls)),
]

## Add our `api` app to the main router file

We now need to add the urls of our API to the main project.

In [ ]:
cat django_metadata_api/urls.py

## Add our api urls

In [ ]:
%%writefile -a django_metadata_api/urls.py

from django.urls import include

urlpatterns.append(path('', include("api.urls")))

## Starting django

Now run

```bash
python manage.py runserver
```

in an external terminal to start the development server and head over to http://127.0.0.1:8000

## Add the parameters to the dataset

In [ ]:
%%writefile -a api/serializers.py


class DatasetSerializer(serializers.HyperlinkedModelSerializer):
    
    parameters = ParameterSerializer(many=True)
    
    class Meta:
        model = models.Dataset
        fields = '__all__'

Checkout the changes at http://127.0.0.1:8000/datasets

## Enable the admin interface

In [ ]:
!cat api/admin.py

In [ ]:
%%writefile api/admin.py

from django.contrib import admin
from api import models


class ParameterInline(admin.TabularInline):
    model = models.Parameter


@admin.register(models.Dataset)
class DatasetAdmin(admin.ModelAdmin):
    """Administration class for the :model:`api.Dataset` model."""

    inlines = [ParameterInline]

    search_fields = ["name", "project"]

## Create a user to access the admin interface

Open a terminal and run

```
python manage.py createsuperuser --email admin@example.com --username admin
```

And checkout http://127.0.0.1:8000/admin

## Restrict PUT and POST to authenticated users

Djangos Rest framework comes with a login and logout functionality that we need to insert into our projects `urls.py` router file.

In [ ]:
%%writefile -a django_metadata_api/urls.py

urlpatterns.insert(
    -2, path('api-auth/', include('rest_framework.urls', namespace='rest_framework'))
)

## Add the permission to our viewsets

In [ ]:
%%writefile -a api/views.py

for view in [PersonViewSet, DatasetViewSet, InstitutionViewSet, ProjectViewSet, ParameterViewSet]:
    view.permission_classes = [permissions.IsAuthenticatedOrReadOnly]

Now you'll see that you cannot make POST requests anymore to http://127.0.0.1:8000/datasets (for instance).

Login at http://127.0.0.1:8000/api-auth/login and it will be possible again.

## Export the schema

Now we can export our database schema to show others, how our RestAPI is structured. For this purpose, we add a new view to our api.

In [ ]:
%%writefile -a api/urls.py

from rest_framework.schemas import get_schema_view

urlpatterns.append(
    path('schema', get_schema_view(
        title="Metadata Portal",
        description="API for retrieving metadata",
        version="1.0.0",
        urlconf='api.urls',
    ), name='openapi-schema'),
)

Head over to http://127.0.0.1:8000/schema to see the results

## The END

That's it. Now you have a well-defined and functional RestAPI with just a few lines of code!